# P2P Lending Analytics

In [1]:
import numpy as np
import pandas as pd

In [2]:
cash_df = pd.DataFrame(columns=['platform_account_name','nonib_cash','ib_cash'])
concat_columns = -20 # no of columns required in the clean concatinated data frame

#### Loanpad

In [3]:
##########################
platform_account_name = 'Loanpad 60d access account'
gross_apr = 0.054 # need to set APR manually for access account
nonib_cash = 2.83 # need to set cash account that loanpad pays interest into. this is non-interest bearing until re-invested to access account
##########################
df = pd.read_csv('data/loanpad.csv')
df['platform_account_name'] = platform_account_name
df['loan_part_name'] = df['Loan ID'].astype('str')
df['loan_part_id'] = df['Loan ID']
df['loan_part_status'] = df['Loan status']
df['loan_part_amt'] = pd.to_numeric(df['Your investment'].str.replace('£', ''))
df['loan_part_matures'] = pd.to_datetime('today') + pd.Timedelta(60, unit='D') # 60 d access account
df['loan_part_starts'] = pd.to_datetime('today') # 60 d access account
df['loan_part_term'] = df['loan_part_matures'] - df['loan_part_starts']
df['loan_part_ttm'] = df['loan_part_matures'] - pd.to_datetime('today')
df['loan_part_blended_ltv'] = pd.to_numeric(df['Loan to value'].str.replace('%', '').str.replace('-', ''))/100
df['loan_part_gross_apr'] = gross_apr
df['loan_part_net_apr'] = df['loan_part_gross_apr'] # loanpad does not charge fees on apr / aer received
loanpad_platf_amt = df['loan_part_amt'].sum()
gbp_apr_per_day = df['loan_part_net_apr']/365*loanpad_platf_amt # loanpad pay APR daily
no_days_to_gbp_10 = pd.to_numeric(10 / gbp_apr_per_day) # loanpad automatically reinvest in increments of GBP 10
df['loan_part_int_per'] = (365 / no_days_to_gbp_10).astype('int')
df['loan_part_gross_aer'] = (1+df['loan_part_net_apr']/df['loan_part_int_per'])**df['loan_part_int_per']-1
df['loan_part_net_aer'] = df['loan_part_gross_aer'] # loanpad does not charge fees on apr / aer received
df['loan_parts_in_loan'] = df['Loan Parts in Loan']
ib_cash = df.loc[df['loan_part_name'] == 'Cash awaiting investment'] # find ib cash value in df
cash_df.loc[len(cash_df)] = { 'platform_account_name': platform_account_name, 'nonib_cash': nonib_cash, 'ib_cash': ib_cash.loan_part_amt.values[0] } # set ib and nonib cash values
df.drop(ib_cash.index.values[0], inplace=True) # drop ib cash value in df
platform_weight = df['loan_part_amt']/( df['loan_part_amt'].sum() + ib_cash.loan_part_amt.values[0] )
df['platform_weighted_net_aer'] = df['loan_part_net_aer']*platform_weight
df['platform_weighted_blended_ltv'] = df['loan_part_blended_ltv']*platform_weight
df['platform_weighted_term'] = df['loan_part_term'] / pd.Timedelta(days=30.44) * platform_weight
df['platform_weighted_ttm'] = df['loan_part_ttm'] / pd.Timedelta(days=30.44) * platform_weight
loanpad_df = df.iloc[:,concat_columns:]
cash_df.to_csv('data/cash.csv')
#loanpad_df.info()
#loanpad_df.loc[0]

#### PropLend

In [4]:
##########################
platform_account_name = 'Proplend'
nonib_cash = 13.79 # need to set cash account that proplend pays interest into. this is non-interest bearing until re-invested to access account
##########################
df = pd.read_csv('data/proplend.csv')
df['platform_account_name'] = platform_account_name
df['loan_part_name'] = df['Loan Name']
df['loan_part_id'] = df['Loan Name'].astype('str')
df['loan_part_status'] = df['Status']
df['loan_part_amt'] = pd.to_numeric(df['Amount'].str.replace(',', ''))
df['loan_part_term'] = df['Duration'].astype('timedelta64[M]')
#df['loan_part_term'] = df['loan_part_matures'] - df['loan_part_starts']
df['loan_part_matures'] = pd.to_datetime(df['End Date'], dayfirst=True)
df['loan_part_starts'] = df['loan_part_matures'] - df['loan_part_term']
df['loan_part_ttm'] = df['loan_part_matures'] - pd.to_datetime('today')
df['loan_part_blended_ltv'] = pd.to_numeric(df['LTV'].str.replace('%', ''))/100
df['loan_part_gross_apr'] = pd.to_numeric(df['Interest Rate'])/100
df['loan_part_net_apr'] = df['loan_part_gross_apr']*.9 # proplend charge 10% fee on gross apr received from borrower
df['loan_part_int_per'] = 12 # proplend roll up interest and pay apr monthly
df['loan_part_gross_aer'] = (1+df['loan_part_net_apr']/df['loan_part_int_per'])**df['loan_part_int_per']-1
df['loan_part_net_aer'] = df['loan_part_gross_aer'] # proplend charge a fee on APR
df['loan_parts_in_loan'] = 1 # proplend do not tranche loans so its always 1 loan part to 1 loan
ib_cash = 0 # PropLend has no interest bearing cash account
cash_df.loc[len(cash_df)] = { 'platform_account_name': platform_account_name, 'nonib_cash': nonib_cash, 'ib_cash': ib_cash } # set ib and nonib cash values
platform_weight = df['loan_part_amt']/( df['loan_part_amt'].sum() + ib_cash )
df['platform_weighted_net_aer'] = df['loan_part_net_aer']*platform_weight
df['platform_weighted_blended_ltv'] = df['loan_part_blended_ltv']*platform_weight
df['platform_weighted_term'] = df['loan_part_term'] / pd.Timedelta(days=30.44) * platform_weight
df['platform_weighted_ttm'] = df['loan_part_ttm'] / pd.Timedelta(days=30.44) * platform_weight
proplend_df = df.iloc[:,concat_columns:]
#proplend_df.info()

#### CrowdProperty

In [5]:
##########################
platform_account_name = 'CrowdProperty AutoInvest'
nonib_cash = 44.80 # need to set cash account that CrowProperty pays interest into. this is non-interest bearing until re-invested to auto-invest account
##########################
df = pd.read_csv('data/crowdproperty.csv')
df['platform_account_name'] = platform_account_name
df['loan_part_name'] = df['Description']
df['loan_part_id'] = df['Bank Reference'].astype('str')
df['loan_part_status'] = df['Status']
df['loan_part_amt'] = pd.to_numeric(df['Amount']).astype('float')
df['loan_part_matures'] = pd.to_datetime(df['Loan End'], format='%d/%m/%Y')
df['loan_part_starts'] = pd.to_datetime(df['Loan Start'], format='%d/%m/%Y')
df['loan_part_term'] = df['loan_part_matures'] - df['loan_part_starts']
df['loan_part_ttm'] = df['loan_part_matures'] - pd.to_datetime('today')
df['loan_part_blended_ltv'] = pd.to_numeric(df['Blended LTV'])
df['loan_part_gross_apr'] = pd.to_numeric(df['Gross APR'])
df['loan_part_net_apr'] =  pd.to_numeric(df['Net APR']) 
df['loan_part_int_per'] = 1 # crowdproperty roll up interest and pay apr annually
df['loan_part_gross_aer'] = (1+df['loan_part_net_apr']/df['loan_part_int_per'])**df['loan_part_int_per']-1
df['loan_part_net_aer'] = df['loan_part_gross_aer'] # crowdproperty charge a spread on APR
df['loan_parts_in_loan'] = df['Loan Parts in Loan']
loan_parts_repaid = df.loc[df['Status'] == 'Paid Back'].index.values
df.drop(loan_parts_repaid, inplace=True) # drop repaid loans from dataframe
ib_cash = 0 # crowdproperty has no interest bearing cash account
cash_df.loc[len(cash_df)] = { 'platform_account_name': platform_account_name, 'nonib_cash': nonib_cash, 'ib_cash': ib_cash } # set ib and nonib cash values
platform_weight = df['loan_part_amt']/( df['loan_part_amt'].sum() + ib_cash )
df['platform_weighted_net_aer'] = df['loan_part_net_aer']*platform_weight
df['platform_weighted_blended_ltv'] = df['loan_part_blended_ltv']*platform_weight
df['platform_weighted_term'] = df['loan_part_term'] / pd.Timedelta(days=30.44) * platform_weight
df['platform_weighted_ttm'] = df['loan_part_ttm'] / pd.Timedelta(days=30.44) * platform_weight
crowdproperty_df = df.iloc[:,concat_columns:]
#crowdproperty_df.info()

#### Invest & Fund

In [6]:
##########################
platform_account_name = 'Invest & Fund'
nonib_cash = 5.58 # need to set cash account that Invest & Fund pays interest into. this is non-interest bearing until re-invested to auto-invest account
##########################
df = pd.read_csv('data/invest_fund.csv')
df['platform_account_name'] = platform_account_name
df['loan_part_name'] = df['Company Name']
df['loan_part_id'] = df['Loan Ref'].astype('str')
df['loan_part_status'] = 'Active'
df['loan_part_amt'] = pd.to_numeric(df['Outstanding Balance']).astype('float')
df['loan_part_term'] = pd.to_timedelta(pd.to_numeric(df['Term Remaining'].str.split('/ ').str[1].str.replace(' days','')), unit='D')
df['loan_part_matures'] = pd.to_datetime(df['Maturity Date'], format='%d-%b-%y')
df['loan_part_starts'] = df['loan_part_matures'] - df['loan_part_term']
df['loan_part_ttm'] = df['loan_part_matures'] - pd.to_datetime('today')
df['loan_part_blended_ltv'] = pd.to_numeric(df['Blended LTV'])/100
df['loan_part_gross_apr'] = np.nan # i&f publish apr as well as aer but don't provide apr in their file
df['loan_part_net_apr'] =  df['loan_part_gross_apr']
df['loan_part_int_per'] = 12 # i&f pay aer with monthly compounding, but the rate in their file is already aer
df['loan_part_gross_aer'] = pd.to_numeric(df['Interest Rate'].str.replace('%', ''))/100 # i&f only provide aer in their file
df['loan_part_net_aer'] = df['loan_part_gross_aer']-0.0075 # i&f charge 0.75% spread on AER
df['loan_parts_in_loan'] = df['Loan Parts in Loan']
ib_cash = 0 # i&f has no interest bearing cash account
cash_df.loc[len(cash_df)] = { 'platform_account_name': platform_account_name, 'nonib_cash': nonib_cash, 'ib_cash': ib_cash } # set ib and nonib cash values
platform_weight = df['loan_part_amt']/( df['loan_part_amt'].sum() + ib_cash )
df['platform_weighted_net_aer'] = df['loan_part_net_aer']*platform_weight
df['platform_weighted_blended_ltv'] = df['loan_part_blended_ltv']*platform_weight
df['platform_weighted_term'] = df['loan_part_term'] / pd.Timedelta(days=30.44) * platform_weight
df['platform_weighted_ttm'] = df['loan_part_ttm'] / pd.Timedelta(days=30.44) * platform_weight
invest_fund_df = df.iloc[:,concat_columns:]
#invest_fund_df.info()

#### Assetz Capital

In [7]:
##########################
platform_account_name = 'Assetz Capital 90d Access'
gross_apr = 0.04 # need to set APR manually for access account. This is the cap, actual could be lower month-to-month.
nonib_cash = 0.0 # need to set cash account that Assetz Capital pays interest and replayments  through the winddown. this is non-interest bearing and needs to be withdrawn.
winddown = pd.to_datetime('31/12/23', dayfirst=True) # expected date of completing winddown
##########################
df = pd.read_csv('data/assetz_capital.csv')
df['platform_account_name'] = platform_account_name
df['loan_part_name'] = df['name']
df['loan_part_id'] = df['id'].astype('str')
df['loan_part_status'] = 'Winddown' # azzetzaptial loan book is in wind down
df['loan_part_amt'] = pd.to_numeric(df['your_total_holding']).astype('float')
df['loan_part_matures'] = winddown
df['loan_part_starts'] = pd.to_datetime('today') # irrelevant as loanbook is in windown
df['loan_part_term'] = df['loan_part_matures'] - df['loan_part_starts']
df['loan_part_ttm'] = df['loan_part_matures'] - pd.to_datetime('today')
df['loan_part_blended_ltv'] = pd.to_numeric(df['ltv'])/100
df['loan_part_gross_apr'] = pd.to_numeric(gross_apr)
df['loan_part_net_apr'] =  df['loan_part_gross_apr'] # assetzcapital cap APR
df['loan_part_int_per'] = 12 # assetzcapital pay APR monthly
df['loan_part_gross_aer'] = (1+df['loan_part_net_apr']/df['loan_part_int_per'])**df['loan_part_int_per']-1
df['loan_part_net_aer'] = df['loan_part_gross_aer'] # assetzcapital pay a fixed/capped apr, no spread on the published apr rate
df['loan_parts_in_loan'] = 1 # assetzcapital generally do not tranche their loans, so its mostly 1 loan part to 1 loan
ib_cash = 0 # assetzcapital has no interest bearing cash account
cash_df.loc[len(cash_df)] = { 'platform_account_name': platform_account_name, 'nonib_cash': nonib_cash, 'ib_cash': ib_cash } # set ib and nonib cash values
platform_weight = df['loan_part_amt']/( df['loan_part_amt'].sum() + ib_cash )
df['platform_weighted_net_aer'] = df['loan_part_net_aer']*platform_weight
df['platform_weighted_blended_ltv'] = df['loan_part_blended_ltv']*platform_weight
df['platform_weighted_term'] = df['loan_part_term'] / pd.Timedelta(days=30.44) * platform_weight
df['platform_weighted_ttm'] = df['loan_part_ttm'] / pd.Timedelta(days=30.44) * platform_weight
assetz_capital_df = df.iloc[:,concat_columns:]
#assetz_capital_df.info()

#### Kuflink

In [8]:
##########################
platform_account_name = 'Kuflink 12m Term 22043'
nonib_cash = 0.0 # need to set cash account, but will typrically be 0 as term account rolls up interest payments until expiry
ltv = 0.6533 # need to set ltv of the pooled account https://invest.kuflink.co.uk/product/pool
gross_apr = 0.05 # need to set APR manually for term account. This is the cap, actual could be lower month-to-month.
term_start = pd.to_datetime('29/07/22', format='%d/%m/%y')
term_finish = pd.to_datetime('29/07/23', format='%d/%m/%y')
##########################
df = pd.read_csv('data/kuflink.csv')
df['platform_account_name'] = platform_account_name
df['loan_part_name'] = df['Loan Name']
df['loan_part_id'] = df['Loan Id'].astype('str')
df['loan_part_status'] = 'Active' # all loans in 12m term will be active
df['loan_part_amt'] = pd.to_numeric(df['Monetary Amount'].str.replace('£', '')).astype('float')
df['loan_part_matures'] = term_finish
df['loan_part_starts'] = term_start
df['loan_part_term'] = df['loan_part_matures'] - df['loan_part_starts']
df['loan_part_ttm'] = df['loan_part_matures'] - pd.to_datetime('today')
df['loan_part_blended_ltv'] = ltv
df['loan_part_gross_apr'] = gross_apr
df['loan_part_net_apr'] =  df['loan_part_gross_apr'] # kuflink term caps APR
df['loan_part_int_per'] = int((term_finish - term_start).days/365) # kuflink term pays interest annually
df['loan_part_gross_aer'] = (1+df['loan_part_net_apr']/df['loan_part_int_per'])**df['loan_part_int_per']-1
df['loan_part_net_aer'] = df['loan_part_gross_aer'] # kuflink pay a fixed/capped apr, no spread on the published apr rate
df['loan_parts_in_loan'] = df['Loan Parts in Loan']
ib_cash = 0 # kuflink has no interest bearing cash account
cash_df.loc[len(cash_df)] = { 'platform_account_name': platform_account_name, 'nonib_cash': nonib_cash, 'ib_cash': ib_cash } # set ib and nonib cash values
platform_weight = df['loan_part_amt']/( df['loan_part_amt'].sum() + ib_cash )
df['platform_weighted_net_aer'] = df['loan_part_net_aer']*platform_weight
df['platform_weighted_blended_ltv'] = df['loan_part_blended_ltv']*platform_weight
df['platform_weighted_term'] = df['loan_part_term'] / pd.Timedelta(days=30.44) * platform_weight
df['platform_weighted_ttm'] = df['loan_part_ttm'] / pd.Timedelta(days=30.44) * platform_weight
kuflink_df = df.iloc[:,concat_columns:]
#kuflink_df.info()

### Concatenate Data

In [9]:
concat_df = pd.concat([loanpad_df, proplend_df, crowdproperty_df, invest_fund_df, assetz_capital_df, kuflink_df], axis=0, ignore_index=True)
concat_df.to_csv('data/concatenated.csv')
cash_df.to_csv('data/cash.csv')
#concat_df.info()

### Enrich Data

In [10]:
concat_df['loan_portion'] = 1/concat_df['loan_parts_in_loan']
portfolio_ib_cash = cash_df['ib_cash'].sum()
portfolio_nonib_cash = cash_df['nonib_cash'].sum()
portfolio_weight = concat_df['loan_part_amt']/(concat_df['loan_part_amt'].sum() + portfolio_ib_cash) # weighting inc. of intrest bearing cash
portfolio_weight_inc_nonib_cash = concat_df['loan_part_amt']/(concat_df['loan_part_amt'].sum() + portfolio_ib_cash + portfolio_nonib_cash) # wighting inc. of interest and non-interest bearing cash
concat_df['portfolio_weighted_net_aer'] = concat_df['loan_part_net_aer']*portfolio_weight
concat_df['portfolio_weighted_net_aer_inc_nonib_cash'] = concat_df['loan_part_net_aer']*portfolio_weight_inc_nonib_cash
concat_df['portfolio_weighted_blended_ltv'] = concat_df['loan_part_blended_ltv']*portfolio_weight
concat_df['portfolio_weighted_blended_ltv_inc_nonib_cash'] = concat_df['loan_part_blended_ltv']*portfolio_weight_inc_nonib_cash
concat_df['portfolio_weighted_term'] = concat_df['loan_part_term'] / pd.Timedelta(days=30.44) * portfolio_weight
concat_df['portfolio_weighted_ttm'] = concat_df['loan_part_ttm'] / pd.Timedelta(days=30.44) * portfolio_weight

status_dict = { 'Extended (No ICF)': 'Active',
                'Active - Good Standing': 'Active',
                'Live': 'Active' }
period_dict = { pd.to_datetime('today'): 'Short Default',
                pd.to_datetime('today') - np.timedelta64(3, 'M'): 'Long Default'}
ttm_arr = [ 'platform_weighted_ttm', 'portfolio_weighted_ttm' ]
term_arr = [ 'platform_weighted_term', 'portfolio_weighted_term' ]
default_criteria = ((concat_df['loan_part_status'] == 'Short Default') 
              | (concat_df['loan_part_status'] == 'Long Default') 
              | (concat_df['loan_part_status'] == 'Winddown'))

for status in status_dict:
    concat_df['loan_part_status'].mask(concat_df['loan_part_status'] == status, status_dict[status], inplace=True)    
for period in period_dict:
    concat_df['loan_part_status'].mask( concat_df['loan_part_matures'] < period, period_dict[period], inplace=True)
for col in ttm_arr: # zeroize ttm for defaulted or it will underweight ttm
    concat_df[col].mask(default_criteria, np.nan, inplace=True)
for col in term_arr: # zeroize weighted term for negative term or it will unerweight term
    concat_df[col].mask(concat_df['loan_part_term'] < pd.Timedelta(0), np.nan, inplace=True)

concat_df.to_csv('data/concatenated.csv')
# concat_df.info()
# correct weights for defaulted loans and negative terms / ttms
# sector
# month maturity